# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('test').getOrCreate()

In [3]:
data = spark.read.csv('customer_churn.csv',header = True,
                    inferSchema = True)

In [6]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [8]:
#prepare data from ml
#no need to encode or index anything since the cols you want to consider--
#-- are all numerical.
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [24]:
required_data = data.select(['Age','Total_Purchase',
                                         'Account_Manager','Years','Num_Sites','Churn'])

In [25]:
#autobots , assemble
from pyspark.ml.feature import VectorAssembler

In [26]:
assembler = VectorAssembler(inputCols = ['Age','Total_Purchase',
                                         'Account_Manager','Years','Num_Sites'],outputCol = 'features')

In [27]:
from pyspark.ml.classification import LogisticRegression

In [28]:
lr = LogisticRegression(featuresCol = 'features',labelCol = 'Churn')

In [29]:
#split into training and testing
train_data,test_data = required_data.randomSplit([0.7,0.3])

In [30]:
from pyspark.ml import Pipeline

In [31]:
pipeline = Pipeline(stages = [assembler,lr])

In [32]:
fitted_model = pipeline.fit(train_data)

In [33]:
results = fitted_model.transform(test_data)

In [37]:
to_evaluate = results.select(['Churn','prediction'])

In [38]:
#get the roc
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [40]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol = 'prediction',
                                           labelCol = 'Churn')

In [41]:
evaluation  = evaluator.evaluate(to_evaluate)

In [42]:
evaluation

0.7508541827399978

In [43]:
new_data = spark.read.csv('new_customers.csv',header = True,
                             inferSchema = True)

In [44]:
new_data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company']

In [45]:
to_predict = new_data.select(['Age','Total_Purchase',
                                         'Account_Manager','Years','Num_Sites'])

In [47]:
new_data_fitted = fitted_model.transform(to_predict)

In [48]:
new_data_fitted

DataFrame[Age: double, Total_Purchase: double, Account_Manager: int, Years: double, Num_Sites: double, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [49]:
new_data_fitted.show()

+----+--------------+---------------+-----+---------+--------------------+--------------------+--------------------+----------+
| Age|Total_Purchase|Account_Manager|Years|Num_Sites|            features|       rawPrediction|         probability|prediction|
+----+--------------+---------------+-----+---------+--------------------+--------------------+--------------------+----------+
|37.0|       9935.53|              1| 7.71|      8.0|[37.0,9935.53,1.0...|[2.01943984996894...|[0.88282307599284...|       0.0|
|23.0|       7526.94|              1| 9.28|     15.0|[23.0,7526.94,1.0...|[-5.8970868712400...|[0.00273990908238...|       1.0|
|65.0|         100.0|              1|  1.0|     15.0|[65.0,100.0,1.0,1...|[-2.4410081015947...|[0.08009860095953...|       1.0|
|32.0|        6487.5|              0|  9.4|     14.0|[32.0,6487.5,0.0,...|[-4.8017912491645...|[0.00814808206978...|       1.0|
|32.0|      13147.71|              1| 10.0|      8.0|[32.0,13147.71,1....|[0.55177550234997...|[0.634547